In [18]:
%load_ext autoreload
%autoreload 2

[autoreload of _pydevd_bundle failed: Traceback (most recent call last):
  File "/Users/realmistic/.local/share/virtualenvs/long-term-news-llm-rag-hx-iPapI/lib/python3.12/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/Users/realmistic/.local/share/virtualenvs/long-term-news-llm-rag-hx-iPapI/lib/python3.12/site-packages/IPython/extensions/autoreload.py", line 475, in superreload
    module = reload(module)
             ^^^^^^^^^^^^^^
  File "/opt/homebrew/Cellar/python@3.12/3.12.7/Frameworks/Python.framework/Versions/3.12/lib/python3.12/importlib/__init__.py", line 130, in reload
    raise ModuleNotFoundError(f"spec not found for the module {name!r}", name=name)
ModuleNotFoundError: spec not found for the module '_pydevd_bundle'
]
[autoreload of _pydev_bundle failed: Traceback (most recent call last):
  File "/Users/realmistic/.local/share/virtualenvs/long-term-news-llm-rag-hx-iPapI/lib/python3.12/site-packages/I

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[autoreload of pydevd failed: Traceback (most recent call last):
  File "/Users/realmistic/.local/share/virtualenvs/long-term-news-llm-rag-hx-iPapI/lib/python3.12/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/Users/realmistic/.local/share/virtualenvs/long-term-news-llm-rag-hx-iPapI/lib/python3.12/site-packages/IPython/extensions/autoreload.py", line 475, in superreload
    module = reload(module)
             ^^^^^^^^^^^^^^
  File "/opt/homebrew/Cellar/python@3.12/3.12.7/Frameworks/Python.framework/Versions/3.12/lib/python3.12/importlib/__init__.py", line 130, in reload
    raise ModuleNotFoundError(f"spec not found for the module {name!r}", name=name)
ModuleNotFoundError: spec not found for the module 'pydevd'
]


In [47]:

from openai import OpenAI
import feedparser
import time

In [4]:
feed_url = 'https://pythoninvest.com/rss-feed-612566707351.xml'
output_file_path = 'data/input_news_feed.json'

feed = feedparser.parse(feed_url)

In [6]:
type(feed) # feed a dictionary


feedparser.util.FeedParserDict

In [8]:
feed['entries'][0].keys()

dict_keys(['title', 'title_detail', 'links', 'link', 'published', 'published_parsed', 'authors', 'author', 'author_detail', 'tags', 'summary', 'summary_detail', 'turbo_content'])

In [10]:
feed['entries'][0]['link']

# we're mostly interested in turbo_content

'https://pythoninvest.com/tpost/yk09rupzv1-week-17-24-july-2023'

In [20]:
# OpenAI client setup with increased timeout
# Read on Ollama - OpenAI compatibility: https://github.com/ollama/ollama/blob/main/docs/openai.md
client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
    timeout = 5 * 60.0  # Increase timeout to 3*60 seconds
)

MAX_RETRIES = 3
RETRY_DELAY = 5
BATCH_SIZE = 1


In [48]:
def llm(prompt, model):
    for attempt in range(MAX_RETRIES):
        try:
            response = client.chat.completions.create(
                model=model,
                temperature=0.0,
                messages=[{"role": "user", "content": prompt}]
            )
            return response
        except Exception as e:
            if attempt < MAX_RETRIES - 1:
                print(f"Attempt {attempt + 1} failed. Retrying in {RETRY_DELAY} seconds...")
                time.sleep(RETRY_DELAY)
            else:
                raise e


In [49]:
# LIST OF CURRENT LOCAL MODELS
!ollama list

NAME                        ID              SIZE      MODIFIED     
phi3.5:latest               3b387c8dd9b7    2.2 GB    7 weeks ago     
mixtral:8x22b               e8479ee1cb51    79 GB     2 months ago    
phi3:14b                    cf611a26b048    7.9 GB    2 months ago    
deepseek-coder-v2:latest    8577f96d693e    8.9 GB    2 months ago    
qwen2:72b                   14066dfa503f    41 GB     2 months ago    
mistral-large:latest        0ca7dfa0bf06    69 GB     2 months ago    
gemma2:27b                  53261bc9c192    15 GB     2 months ago    
llama3:70b                  786f3184aec0    39 GB     2 months ago    
llama3.1:latest             91ab477bec9d    4.7 GB    2 months ago    


In [50]:
# example calling a local LLM via OpenAI compatibility
res = llm(prompt = "Tell me a joke",model="llama3.1:latest")
print(res.choices[0].message.content)

Here's one:

What do you call a fake noodle?

An impasta!

Hope that made you laugh! Do you want to hear another one?


* Inspired by this code:
https://github.com/alexeygrigorev/fitness-assistant/blob/main/notebooks/evaluation-data-generation.ipynb

In [43]:
prompt_template = '''TASK: Extract all individual news summary from the html file and RETURN THE JSON FILE CONTENTS, not code.
 EXAMPLE: 
...
 NEWS SUMMARY for ('TSLA', 42), which changed on 2.43% last trading day:
<some text>.
....

needs to be converted to this section of JSON: "type":'individual', "ticker":'TSLA', "count":42, "last_trading_day_growth":2.43, "news_summary":<some text>

You will have a few entries to be extracted as an array of dictionaries and saved to JSON. 

Real text to extract from:
{news_section}
'''

In [51]:
# Now use LLM properly: asking to scrape the 
news_content = feed['entries'][0]['turbo_content']
prompt = prompt_template.format(news_section=news_content)

print(prompt)

TASK: Extract all individual news summary from the html file and RETURN THE JSON FILE CONTENTS, not code.
 EXAMPLE: 
...
 NEWS SUMMARY for ('TSLA', 42), which changed on 2.43% last trading day:
<some text>.
....

needs to be converted to this section of JSON: "type":'individual', "ticker":'TSLA', "count":42, "last_trading_day_growth":2.43, "news_summary":<some text>

You will have a few entries to be extracted as an array of dictionaries and saved to JSON. 

Real text to extract from:
<header><h1>Week 17-24 July 2023</h1></header><figure><img src="https://static.tildacdn.com/tild6131-3163-4135-a565-356661323733/the-broad-industry-o.svg"/></figure><div class="t-redactor__embedcode"><script async src="https://pagead2.googlesyndication.com/pagead/js/adsbygoogle.js?client=ca-pub-5845276189467216"
     crossorigin="anonymous"></script></div><div class="t-redactor__text"><strong>DISCLAIMER</strong><br />The content provided below was generated by AI (OpenAI's ChatGPT) using titles and descri

Ivan: YOU CAN INSERT THE PROMPT ABOVE TO THE chatgpt.com UI and check that it returns the correct stats like: 
```json
[
  {
    "type": "individual",
    "ticker": "TSLA",
    "count": 42,
    "last_trading_day_growth": 2.43,
    "news_summary": "text"
  },
  ...
]
```

In [52]:
# Let's try it locally
scraped_data = llm(prompt = prompt, model="llama3:70b")

# Result: 
scraped_data.choices[0].message.content

'Here is the JSON file contents with individual news summaries:\n\n```\n[\n    {\n        "company": "Pentair",\n        "summary": "Performance in the second quarter is expected to be affected by inflated costs and supply chain issues."\n    },\n    {\n        "company": "Robert Half",\n        "summary": "Second-quarter earnings and revenues are expected to decline sequentially."\n    },\n    {\n        "company": "VeriSign",\n        "summary": "Second-quarter performance is expected to benefit from continued growth in domain name registrations."\n    },\n    {\n        "company": "AMC",\n        "summary": "The court ruling on preferred equity shares could take the company back to the drawing board."\n    },\n    {\n        "company": "Cloros Mortgage Trust",\n        "summary": "Upgraded from underweight to neutral by JP Morgan."\n    },\n    {\n        "company": "Galmed Pharmaceuticals",\n        "summary": "Upgraded from hold to buy by Maxim Group."\n    },\n    {\n        "com

In [54]:
print(scraped_data.choices[0].message.content)

Here is the JSON file contents with individual news summaries:

```
[
    {
        "company": "Pentair",
        "summary": "Performance in the second quarter is expected to be affected by inflated costs and supply chain issues."
    },
    {
        "company": "Robert Half",
        "summary": "Second-quarter earnings and revenues are expected to decline sequentially."
    },
    {
        "company": "VeriSign",
        "summary": "Second-quarter performance is expected to benefit from continued growth in domain name registrations."
    },
    {
        "company": "AMC",
        "summary": "The court ruling on preferred equity shares could take the company back to the drawing board."
    },
    {
        "company": "Cloros Mortgage Trust",
        "summary": "Upgraded from underweight to neutral by JP Morgan."
    },
    {
        "company": "Galmed Pharmaceuticals",
        "summary": "Upgraded from hold to buy by Maxim Group."
    },
    {
        "company": "Wells Fargo",
        

In [ ]:
# TODO: DEBUG and learn how to read and store content in some array like: 
# replace "content" : null TO
# "content" : [{
#   {"type":'individual',
#    "ticker":'NVDA',
#    "text":<summary_text>}, 
#   ...., <all 1 ticker summaries>
#   {"type":'market_1day',
#    "model" :"gpt3.5 or gpt4"  
#    "ticker":'many',
#    "text":<summary_text>}, 
#  {"type":'market_1week',
#    "model" :"gpt3.5 or gpt4"  
#    "ticker":'many',
#    "text":<summary_text>} 
#  ]
